<a href="https://colab.research.google.com/github/Elizav555/Data_Mining/blob/main/Distances/Distances.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import nltk
import requests
import string
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Сначала я решила взять статью https://www.thegamer.com/horizon-forbidden-west-review-ps5/, которую трудно отнести к какой-либо категории, но мне был интересен результат 

In [5]:
url = 'https://www.thegamer.com/horizon-forbidden-west-review-ps5/'
headers = requests.utils.default_headers()
headers.update(
    {
        'User-Agent': 'My User Agent 1.0',
    }
)
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text)
root = soup.body
root_childs = [e.name for e in root.descendants if e.name is not None]
#print(set(root_childs))

В коде выше я просматривала список Html тэгов, которые содержат текст. Ниже я получила все слова, которые они содержат

In [26]:
textTags = soup.find_all(['p','h1','h2','h3','h4','label','a','ul','li'])
whole_text = ' '.join(map(lambda it: it.text, textTags)) 
all_words = word_tokenize(whole_text)
#print(all_words)
print('Number of all words: ',len(all_words))

Number of all words:  2636


Функцию ниже я использовала, чтобы очищать текст от знаков пунктуации, стоп слов и цифр как при обработке статей, так и при составлении условных словарей, которые содержали слова, относящиеся к категориям

In [27]:
def process_whole_text(all_words_set):
  symb_to_remove = stopwords.words('english') +  list(string.punctuation) +  list(string.digits) + list('’')
  text_cleared = [word for word in all_words_set if word not in symb_to_remove]
  text_cleared = [word for word in text_cleared if not word.isdigit()]
  #text_cleared = set(text_cleared)
  #print(text_cleared)
  return text_cleared

horizon_article = process_whole_text(all_words)
print('Number of cleared words: ',len(horizon_article))

Number of cleared words:  1642


https://keywordgenerator.net

Сервис, откуда я взяла слова для топиков

In [28]:
def process_key_words(file_name, key ):
  dataset = pd.read_csv(file_name)
  dataset = dataset[key]
  words = dataset.values.tolist()
  words = list(map(lambda it: it.split(' ') ,words))
  words_set = list()
  for words in words:
    words_set+=words
  words_set = set(words_set)
  words_set-=set(['']) 
  words_set = process_whole_text(words_set)
  return set(words_set)

In [11]:
news_words = process_key_words('news.csv','Keyword')
science_words = process_key_words('science.csv','Keyword')
shopping_words = process_key_words('shopping.csv','Keyword')
sport_words = process_key_words('sport.csv','Keyword')

news_words = news_words.union(process_key_words('news2.csv','Keywords'))
science_words = science_words.union(process_key_words('science2.csv','Keywords'))
shopping_words = shopping_words.union(process_key_words('shopping2.csv','Keywords'))
sport_words = sport_words.union(process_key_words('sport2.csv','Keywords'))
# print('News: ',news_words,'\n', len(news_words))
# print('Science: ',science_words,'\n', len(science_words))
# print('Shopping: ',shopping_words,'\n', len(shopping_words))
# print('Sport: ',sport_words,'\n', len(sport_words))
print('News: ', len(news_words))
print('Science: ', len(science_words))
print('Shopping: ', len(shopping_words))
print('Sport: ', len(sport_words))

News:  983
Science:  1089
Shopping:  1072
Sport:  1052


In [12]:
def parse_from_dictionary(url):
  response_dictionary = requests.get(url)
  soup_dictionary = BeautifulSoup(response_dictionary.text)
  textTags_dictionary = soup_dictionary.find_all('a',class_='btnpopover')
  whole_text_dictionary = ' '.join(map(lambda it: it.text, textTags_dictionary)) 
  all_words_dictionary = word_tokenize(whole_text_dictionary)
  all_words_dictionary = process_whole_text(all_words_dictionary)
  return(set(all_words_dictionary))


https://www.niftyword.com/related-words/science/ связанные слова

In [13]:
news_words = news_words.union(parse_from_dictionary('https://www.niftyword.com/related-words/news/'))
science_words = science_words.union(parse_from_dictionary('https://www.niftyword.com/related-words/science/'))
shopping_words= shopping_words.union(parse_from_dictionary('https://www.niftyword.com/related-words/shopping/'))
sport_words =sport_words.union(parse_from_dictionary('https://www.niftyword.com/related-words/sport/'))
# print('News: ',news_words,'\n', len(news_words))
# print('Science: ',science_words,'\n', len(science_words))
# print('Shopping: ',shopping_words,'\n', len(shopping_words))
# print('Sport: ',sport_words,'\n', len(sport_words))
print('News: ', len(news_words))
print('Science: ', len(science_words))
print('Shopping: ', len(shopping_words))
print('Sport: ', len(sport_words))

News:  2756
Science:  3666
Shopping:  3710
Sport:  3403


https://keywordtool.io/ru/search/keywords/google/

еще один сервис, в котором искала ключевые слова, теперь я могла приступать к анализу первой статьи

In [14]:
def jaccard_fun(a,b,topic):
  a= set(a)
  b=set(b)
  shared = a.intersection(b)
  total = a.union(b)
  res = len(shared) / len(total)
  print('Topic: ',topic,', Shared: ',len(shared),', Total: ',len(total),', Similarity: ',res,', Distance: ',1-res)

In [15]:
jaccard_fun(horizon_article ,news_words,'News')
jaccard_fun(horizon_article ,science_words,'Science')
jaccard_fun(horizon_article ,shopping_words,'Shopping')
jaccard_fun(horizon_article ,sport_words,'Sport')

Topic:  News , Shared:  99 , Total:  3394 , Similarity:  0.029169121979964643 , Distance:  0.9708308780200353
Topic:  Science , Shared:  116 , Total:  4287 , Similarity:  0.027058549101936086 , Distance:  0.9729414508980639
Topic:  Shopping , Shared:  121 , Total:  4326 , Similarity:  0.027970411465557097 , Distance:  0.9720295885344429
Topic:  Sport , Shared:  126 , Total:  4014 , Similarity:  0.03139013452914798 , Distance:  0.968609865470852


Первая выбранная мною статья, оцененная с помощью **коэф жаккара** **оказалась на примерно равном расстоянии** от всех категорий - **приблизительно 0,97**. Однако коэфициент схожести для категорий **новости** и **спорт** были чуть выше, что я считаю верным, так как материал статьи это ревью на новую компьютерную игру

https://www.topendsports.com/world/lists/popular-sport/ - статья про популярные виды спорта оказалась моей следующей жертвой

In [16]:
response_dictionary = requests.get('https://www.topendsports.com/world/lists/popular-sport/')
soup_dictionary = BeautifulSoup(response_dictionary.text)
textTags_dictionary = soup_dictionary.find_all(['p','h1','li','a','h2'])
whole_text_dictionary = ' '.join(map(lambda it: it.text, textTags_dictionary)) 
all_words_dictionary = word_tokenize(whole_text_dictionary)
#print(all_words_dictionary)
print('Number of all words: ',len(all_words_dictionary))

Number of all words:  16194


In [17]:
sport_article = process_whole_text(all_words_dictionary)
#print(sport_article)
print('Number of cleared words: ',len(sport_article))

Number of cleared words:  8502


In [18]:
jaccard_fun(sport_article ,news_words,'News')
jaccard_fun(sport_article ,science_words,'Science')
jaccard_fun(sport_article ,shopping_words,'Shopping')
jaccard_fun(sport_article ,sport_words,'Sport')

Topic:  News , Shared:  338 , Total:  5310 , Similarity:  0.06365348399246705 , Distance:  0.9363465160075329
Topic:  Science , Shared:  405 , Total:  6153 , Similarity:  0.06582155046318869 , Distance:  0.9341784495368113
Topic:  Shopping , Shared:  433 , Total:  6169 , Similarity:  0.07018965796725564 , Distance:  0.9298103420327444
Topic:  Sport , Shared:  509 , Total:  5786 , Similarity:  0.08797096439681991 , Distance:  0.91202903560318


Здесь **коэф жаккара** определил, что наименьшее растояние между этой статьей и тематикой **спорта**, однако из-за большой разницы в размерых словарей и кол-ва слов из статьи схожесть все еще невелика.

In [19]:
data = [" ".join(horizon_article), " ".join(sport_article)," ".join(news_words)," ".join(science_words)," ".join(shopping_words)," ".join(sport_words)]
count_vectorizer = CountVectorizer()
vector_matrix = count_vectorizer.fit_transform(data)
tokens = count_vectorizer.get_feature_names_out()
tokens

array(['000', '031', '0n', ..., 'zymology', 'zymurgy', 'špansko'],
      dtype=object)

In [20]:
vector_matrix_array = vector_matrix.toarray()
vector_matrix_array

array([[0, 0, 0, ..., 0, 0, 0],
       [3, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 1, ..., 0, 0, 0]])

In [21]:
def create_dataframe(matrix, tokens):
    doc_names = ["horizon_article","sport_article","news_words","science_words","shopping_words","sport_words"]
    df = pd.DataFrame(data=matrix, index=doc_names, columns=tokens)
    return(df)

dataframe = create_dataframe(vector_matrix_array,tokens)
dataframe

,000,031,0n,0nline,0pera,0t,10,100,10th,11,...,zoom,zoos,zourkhaneh,zp,zulla,zuoz,zurich,zymology,zymurgy,špansko
horizon_article,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
sport_article,3,0,0,0,0,0,2,2,0,0,...,0,0,0,0,0,0,0,0,0,0
news_words,0,0,1,0,0,0,0,0,0,0,...,1,1,0,1,0,0,1,0,0,0
science_words,0,1,0,0,0,0,1,0,1,1,...,1,0,0,0,0,0,0,1,1,0
shopping_words,0,0,0,1,0,0,0,0,0,3,...,0,0,0,0,1,0,1,0,0,1
sport_words,0,0,1,0,1,1,0,1,0,0,...,0,0,1,0,0,1,0,0,0,0


Тогда я решила попробовать представить свои сеты данных в виде векторов, где 1 обозначает, что данное слово присутствует в наборе (больше 1 если несколько раз), а 0 при его отсутствии

In [22]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity_matrix = cosine_similarity(vector_matrix)
create_dataframe(cosine_similarity_matrix,["horizon_article","sport_article","news_words","science_words","shopping_words","sport_words"])

,horizon_article,sport_article,news_words,science_words,shopping_words,sport_words
horizon_article,1.000000,0.261103,0.078373,0.067690,0.075192,0.078453
sport_article,0.261103,1.000000,0.074702,0.075010,0.074599,0.143240
news_words,0.078373,0.074702,1.000000,0.195748,0.285372,0.189667
science_words,0.067690,0.075010,0.195748,1.000000,0.242570,0.272381
shopping_words,0.075192,0.074599,0.285372,0.242570,1.000000,0.338550
sport_words,0.078453,0.143240,0.189667,0.272381,0.338550,1.000000


Метрики **косинуса** дали результаты, которые представлены выше. В целом также как и для коэф жаккара **для статьи обзора игры** дистанция до тематик **спорт** и **новости** оказались наименьшими и примерно одинаковыми около 0,92

Для статьи про спорт результаты оказались **более показательными**, так как разница в схожести между спортом и остальными категориями увеличилась: 
**для спорта расстояние примерно 0,85** , когда для остальных категорий оно приближено к 0,92

In [23]:
def jaccard_vector_fun(x, y,topic):
    intersect = 0
    union = 0
    for i,val in enumerate(x):
      if x[i]&y[i]!=0:
        intersect +=1
      if x[i]|y[i]!=0:
        union +=1
    res = intersect / union
    print('Topic: ',topic,', Shared: ',intersect,', Total: ',union,', Similarity: ',res,', Distance: ',1-res)

In [24]:
jaccard_vector_fun(vector_matrix_array[0] ,vector_matrix_array[2],'News')
jaccard_vector_fun(vector_matrix_array[0] ,vector_matrix_array[3],'Science')
jaccard_vector_fun(vector_matrix_array[0],vector_matrix_array[4],'Shopping')
jaccard_vector_fun(vector_matrix_array[0],vector_matrix_array[5],'Sport')

Topic:  News , Shared:  97 , Total:  3273 , Similarity:  0.029636419187289948 , Distance:  0.97036358081271
Topic:  Science , Shared:  99 , Total:  4162 , Similarity:  0.02378664103796252 , Distance:  0.9762133589620374
Topic:  Shopping , Shared:  118 , Total:  4184 , Similarity:  0.02820267686424474 , Distance:  0.9717973231357553
Topic:  Sport , Shared:  113 , Total:  3891 , Similarity:  0.029041377537907993 , Distance:  0.970958622462092


In [25]:
jaccard_vector_fun(vector_matrix_array[1] ,vector_matrix_array[2],'News')
jaccard_vector_fun(vector_matrix_array[1] ,vector_matrix_array[3],'Science')
jaccard_vector_fun(vector_matrix_array[1],vector_matrix_array[4],'Shopping')
jaccard_vector_fun(vector_matrix_array[1],vector_matrix_array[5],'Sport')

Topic:  News , Shared:  254 , Total:  4683 , Similarity:  0.054238735853085626 , Distance:  0.9457612641469144
Topic:  Science , Shared:  268 , Total:  5543 , Similarity:  0.04834926934872812 , Distance:  0.9516507306512719
Topic:  Shopping , Shared:  301 , Total:  5545 , Similarity:  0.054283137962128046 , Distance:  0.945716862037872
Topic:  Sport , Shared:  364 , Total:  5167 , Similarity:  0.07044706793110123 , Distance:  0.9295529320688988


Выше я проверила, будет ли более эффективным применение **коэф жаккара** для **векторов**, но результаты не сильно изменились